In [110]:
import numpy as np
from PIL import Image
from PIL import ImageDraw
import netCDF4
import os

In [111]:
input_folder = "/mnt/hdd/eric/.tmp_ipy/00.Data/NC/20230501"
idx= 3

[ i for i in os.listdir(input_folder) if i.__contains__(".nc")]

['GK2_GOCI2_L1B_20230501_011530_LA_S007.nc',
 'GK2_GOCI2_L1B_20230501_001530_LA_S007.nc',
 'GK2_GOCI2_L1B_20230501_031530_LA_S007.nc',
 'GK2_GOCI2_L1B_20230501_021530_LA_S007.nc']

In [112]:
fname = os.path.join( input_folder, [ i for i in os.listdir(input_folder) if i.__contains__(".nc")][idx] )
end_fix = fname.split("/")[-1].replace(".nc",".png")
outpath = os.path.join( input_folder, end_fix) 

In [113]:
def read_refl_fr_LA(fname):
    nc = netCDF4.Dataset(fname)
    wavestr=['380','412','443','490','510','555','620','660','680','709','745','865']
    for ib, wavenm in enumerate(wavestr):
        bdstr='L_TOA_'+wavenm
        _=nc.groups['geophysical_data'][bdstr]
        #print(f0_band[ib])
        _ref=_[:].data#/mu0[:]*math.pi/f0_band[ib]*0.1
        _mask=_[:].mask
        _ref[_mask]=0.
        _ref=np.float32(_ref) # float64->32
        if ib==0:
            rhorc = [_ref]
        else:
            rhorc = np.concatenate((rhorc, [_ref]))
    nc.close()
    return rhorc

In [114]:
bds = [7,5,2]
rhorc = read_refl_fr_LA(fname)
L_TOA = np.transpose(rhorc,(1,2,0))
nbins = 5000

In [115]:
for count, ib in enumerate(bds):
    imgsub = L_TOA[:,:,ib]
    mx=np.amax(imgsub); mn=np.amin(imgsub)
    datarange=(mn+(mx-mn)*0.0, mx-(mx-mn)*0.0) #5% to 95%
    hist, edges=np.histogram(imgsub,bins=nbins,range=datarange)
    cdf = np.cumsum(hist)
    cdf=np.insert(cdf,0,0)
    minv=edges[0]; maxv=edges[-1]
    img_ = ((imgsub-minv)/(maxv-minv)*nbins).astype('int')
    img_[img_ <0]=0 ; img_[img_>nbins]=nbins
    img_=cdf[img_]

    formatted = (img_ * 255 / np.max(img_)).astype('uint8')
    if count == 0: red = Image.fromarray(formatted)
    if count == 1: green = Image.fromarray(formatted)
    if count == 2: blue = Image.fromarray(formatted)
    count=count+1
    del imgsub

out = Image.merge("RGB", (red, green, blue))
draw = ImageDraw.Draw(out)

out.save(outpath)
